# Doppelspaltexperiment visualisiert

Wir nehmen an, dass wir die grundlegenden physikalischen Parameter haben:

1. **Wellenlänge des Lichts ($λ$)**.
2. **Abstand zwischen den Spalten ($d$)**.
3. **Entfernung zwischen den Spalten und dem Schirm ($L$)**.
4. **Positionen auf dem Schirm ($x$)**.

Das Interferenzmuster entsteht aufgrund der Überlagerung der Wellen von den beiden Spalten, wobei sich konstruktive und destruktive Interferenz einstellen. Die Formel für die Position der Maxima lautet:

$$
y_m = m \cdot \frac{\lambda L}{d}
$$

wobei:
- $y_m$ die Position des m-ten Maximums ist.
- $m$ die Ordnung des Maximums ist (z. B. m = 0 für das zentrale Maximum).
- $\lambda$ die Wellenlänge ist.
- $L$ die Entfernung zum Schirm ist.
- $d$ der Abstand zwischen den Spalten ist.

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Physikalische Parameter
wellenlaenge = 500e-9  # Wellenlänge in Metern (z. B. 500 nm für grünes Licht)
abstand_spalten = 1e-4  # Abstand zwischen den Spalten in Metern
entfernung_schirm = 1.0  # Entfernung zum Schirm in Metern
max_ordnungen = 10  # Maximale Anzahl der zu zeigenden Maxima

# Berechnung der Positionen der Maxima
m_values = np.arange(-max_ordnungen, max_ordnungen + 1)
maxima_positions = m_values * (wellenlaenge * entfernung_schirm) / abstand_spalten

# Visualisierung
fig = go.Figure()

# Plot des Interferenzmusters
fig.add_trace(go.Scatter(
    x=maxima_positions, 
    y=np.zeros_like(maxima_positions), 
    mode='markers',
    marker=dict(symbol='x', color='red', size=10),
    name='Maxima'
))

# Plot der theoretischen Wellenform (Interferenzmuster)
x_vals = np.linspace(-0.05, 0.05, 500)  # Bereich des Schirms
intensitaet = (np.cos(np.pi * abstand_spalten * x_vals / (wellenlaenge * entfernung_schirm)))**2  # Intensitätsverteilung

fig.add_trace(go.Scatter(
    x=x_vals, 
    y=intensitaet, 
    mode='lines', 
    name='Interferenzmuster'
))

# Layout-Optionen
fig.update_layout(
    title="Doppelspaltexperiment: Interferenzmuster",
    xaxis_title="Position auf dem Schirm (m)",
    yaxis_title="Intensität",
    template="plotly_dark",
    showlegend=True
)

# Anzeige des Plots
fig.show()


In [ ]:
import dash
from dash import html
import dash_deck
import pydeck

app = dash.Dash(__name__)

# GeoJSON-Daten-URL (kostenlos)
DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"

# Erstelle einen Polygon-Layer
polygon = pydeck.Layer(
    "PolygonLayer",
    [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]],
    stroked=False,
    get_polygon="-",
    get_fill_color=[0, 0, 0, 20],
)

# Erstelle einen GeoJSON-Layer mit Extrusion
geojson = pydeck.Layer(
    "GeoJsonLayer",
    DATA_URL,
    extruded=True,
    get_elevation="properties.valuePerSqm / 20",
    get_fill_color="[255, 255, properties.growth * 255]",
)

# Setze den initialen ViewState
INITIAL_VIEW_STATE = pydeck.ViewState(
    latitude=49.254,
    longitude=-123.13,
    zoom=11,
    max_zoom=16,
    pitch=45,
    bearing=0
)

# Erstelle das Deck mit einem kostenlosen Basemap-Style (z.B. Carto Positron)
r = pydeck.Deck(
    layers=[polygon, geojson],
    initial_view_state=INITIAL_VIEW_STATE,
    map_style="https://basemaps.cartocdn.com/gl/positron-gl-style/style.json"
)

# Erstelle das Dash-DeckGL-Komponenten-Element. Der Parameter "mapboxKey" bleibt leer.
deck_component = dash_deck.DeckGL(
    r.to_json(),
    id="deck-gl",
    mapboxKey=""
)

# Definiere das Layout der Dash-App
app.layout = html.Div(deck_component)

if __name__ == "__main__":
    app.run(debug=True)
